<a href="https://colab.research.google.com/github/jesusGS01/TGINE/blob/main/3_1_scrapyRSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 3 - Scrapy
En esta sesión se va a ver una breve introducción a cómo se pueden crear Crawlers sencillos con la librería Scrapy de cara a hacer la práctica 1.

Lo primero que haremos será instalarnos la librería

In [ ]:
!pip3 install -U scrapy

## Apartado 1.1 Crawler de ficheros RSS
Para desarrollar un Crawler en Scrapy debemos definir una clase Spider que herede de scrapy.Spider. Cada clase definirá los métodos necesarios para realizar el scrapping del sitio web y cómo obtener información estructurada.

Para poder crear Spiders adecuados es NECESARIO conocerse los selectores CSS de HTML.
https://www.w3schools.com/cssref/css_selectors.asp

En el ejemplo de abajo está comentado cómo se haría el scrapping the un fichero en RSS.

In [ ]:
import scrapy
import sys
import json
import locale
import time
import string
import random
from bs4 import BeautifulSoup

class RSSSpider (scrapy.Spider):

    # Es obligatorio poner el nombre del Spider
    name = 'RSS'

    # Estas son las URLs donde empieza a buscar y se podrían poner varias distintas de RSS feeds
    start_urls = ['https://www.abc.es/rss/2.0/portada/',
                  'https://www.laverdad.es/rss/2.0/portada',
                  'https://www.europapress.es/rss/rss.aspx',
                  'https://feeds.elpais.com/mrss-s/pages/ep/site/elpais.com/portada']

    # para evitar que el sitio te bloquee por usar scrapy es interesante cambiar el USER_AGENT
    # El user agent por defecto de Scrapy cuando hace una petición es
    # Scrapy/VERSION (+https://scrapy.org)
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }


    def parse (self, response):
        """
        @inherit

        @param self
        @param response
        """

        #Buscamos todos los elementos en el archivo XML con la etiqueta <item>
        for item in response.css ('XXXXXX'):
            #Obtenemos por cada elemento <item> el texto del subelemento <link>
            url = str (item.css ('XXXXXX').get ()).strip()
            #Obtenemos por cada elemento <item> el texto del subelemento <title>. Además co el BeautifulSoup
            #procesamos el texto en html y nos quedamos con el texto
            title = BeautifulSoup(str(item.css ('XXXXXXXX').get()), 'html.parser').get_text().strip()
            #Obtenemos por cada elemento <item> el texto del subelemento <title>
            content = BeautifulSoup(str(item.css ('XXXXXXXXXX').get()), 'html.parser').get_text().strip()

            #Imprimimos la información obtenida para comprobar lo que estamos extrayendo
            print ("-------------------------")
            print ('URL:' + url)
            print ('Título:' + title)
            print ('Descripción:' + content)
            print ("-------------------------")

            data = {
                'url' : url,
                'title': title,
                'content': content
            }

            #Creamos para cada item un fichero json y para ello obtenemos un número aleatorio.
            filename = str(random.random()).replace(".","") + ".json"

            # Si tenemos descripción, url y título entonces lo guardamos a disco en la carpeta 'rss'
            if content and title and url:
                with open ('rss/' + filename, 'w') as f:
                    json.dump (data, f, indent = 4)

## Apartado 1.2 - Lanzar el Crawler RSSSpyder
Para poder lanzar el Spider necesitamos que ejecutar el siguiente código donde se configuará y lanzará el proceso.
Hay que hacer notar que solamente se puede lanzar un proceso por cada sesión en Jupyter notebook es por eso por lo que se recomienda exportar el código en un script de Python .py para poder ejecutarlo desde la línea de comandos.

In [ ]:
import os
import scrapy
from scrapy.crawler import CrawlerProcess

# Creamos un proceso de Crawler podemos poner distintas settings que están definidas en la documentación.
# Entre ellas podemos ocular los logs del proceso de Crawling.
process = CrawlerProcess(settings={
    "LOG_ENABLED": False,
    # Used for pipeline 1
})

# Como se ha definido anteriormente en el RSSCrawler, los ficheros se van a almacenar en la carpeta "rss"
# Comprobamos que existe la carpeta y si no existe la creamos
if (not os.path.exists('rss')):
    os.mkdir('rss')

# Creamos el proceso con el RSSSpider
process.crawl(RSSSpider)
# Ejecutamos el Crawler
process.start()